In [18]:
%matplotlib inline

import mido
import numpy as np
from math import ceil
import keras
import random
import music21
import glob
import os
import pickle

In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

In [3]:

#Args: mido.mido_obj
# output notelist


In [4]:
def get_notelist(mido_obj, res=8, note_range=(36,84)):
    """
    Args: 
        res (int) - resolution.  8 is eighth notes, 4 quarter, etc...
        note_range (tuple) - middle C = 60
    """

    # this gives # of ticks in a column of the piano sheet
    tick_step = round(float(mido_obj.ticks_per_beat/res))

    active_notes = [0] * abs(np.diff(note_range)[0])

    counter = 0
    notelist = []

    rawmsgs = [msg for track in mido_obj.tracks for msg in track]

    # have to reset counter when a new track is detected
    for msg in rawmsgs:
        if msg.type not in set(['note_on','note_off']):
            continue

        #counter += ceil(msg.time/tick_step)
        counter += msg.time

        if msg.type == 'note_on' and msg.velocity > 0:        
            active_notes[msg.note - note_range[0]] = counter

        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            # fill everything up to this with 1s
            start = active_notes[msg.note - note_range[0]]  #round this section

            notelist.append((msg.note, int(ceil(start/tick_step)), int(ceil((counter-start)/tick_step))))
            active_notes[msg.note - note_range[0]] = 0

    notelist = sorted(notelist, key=lambda x: x[1])
    return (counter, notelist)

In [1]:
def make_musicmat(notelist, steps, note_range=(36,84)):
    """ turns a notelist into a musicmat object
    """
    music_mat = np.zeros([steps, int(np.diff(note_range))])
    cnt = 0
    for nt in notelist:
        c = range(nt[1], (nt[2] + nt[1]))
        r = [nt[0] - note_range[0]] * len(c)
        music_mat[c,r] = np.array([1] * len(r) )
        cnt += 1
    return music_mat

In [2]:
def segmentize(musicmat, maxlen, step):
    """ Converts a musicmat matrix into 
    a 3D matrix with slices of length maxlen
    """
    X = []
    y = []
    steps = musicmat.shape[0]
    for ix in range(0, steps-maxlen-1, step):
        end = ix + maxlen
        X.append(musicmat[ix:end:1, :])
        y.append(musicmat[end+1,:])
    X = np.stack(X)
    y = np.stack(y)
    return X,y

In [7]:
#params

res = 8
note_range=(36,84)

In [8]:
raw_data = []

guitar_dir = 'data/tr_guitar_licks/'
for fname in glob.glob(guitar_dir + '*'):
    try:
        midifile = mido.MidiFile(fname)
        counter, notelist = get_notelist(midifile)
        tick_step = round(float(midifile.ticks_per_beat/res))
        steps = int(ceil(counter/tick_step))
        music_mat = make_musicmat(notelist, steps)
        raw_data.append(music_mat)
    except:
        pass

### Machine readable inputs

In [9]:
maxlen = 50

X = []
y = []
for sample in raw_data:
    smpX, smpy = segmentize(sample, maxlen, 1)
    X.append(smpX)
    y.append(smpy)

X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)

### Train the model

In [10]:
nnotes = np.diff(note_range)[0]
maxlen = 50
step = 1

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, nnotes)))
model.add(Dense(nnotes))
model.add(Dense(nnotes))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, nnotes)))
model.add(Dense(nnotes))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
#X = X.astype(bool)
model.fit(X,y, batch_size=50, nb_epoch=10)

Epoch 1/10
16989/16989 [==============================] - 16s - loss: 4.2758    
Epoch 2/10
16989/16989 [==============================] - 15s - loss: 3.7675    
Epoch 3/10
16989/16989 [==============================] - 15s - loss: 3.5406    
Epoch 4/10
16989/16989 [==============================] - 15s - loss: 3.3112    
Epoch 5/10
16989/16989 [==============================] - 15s - loss: 3.1039    
Epoch 6/10
16989/16989 [==============================] - 15s - loss: 2.9127    
Epoch 7/10
16989/16989 [==============================] - 15s - loss: 2.7596    
Epoch 8/10
16989/16989 [==============================] - 15s - loss: 2.6361    
Epoch 9/10
16989/16989 [==============================] - 15s - loss: 2.5277    
Epoch 10/10
16989/16989 [==============================] - 15s - loss: 2.4448    


In [14]:
model_json = model.to_json()

In [16]:
with open('model.json','w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

In [21]:
pickle.dump(X, open('X.p', 'wb'))